AIDev Dataset Analysis Pipeline

 **Research Goal**: Analyze the quality and maintainability characteristics of AI-generated

 **Research Questions**:
 - RQ1: How do AI-generated code contributions differ from human contributions in structural quality?
 - RQ2: What is the relationship between AI-generated code and post-merge maintenance effort?
 - RQ3: Which characteristics of Agentic-PRs predict successful integration?

 **Dataset**: AIDev-pop (7,122 Agentic-PRs from 856 repos >500 stars) + Human-PRs baseline



In [ ]:
# Install dependencies
import sys
!{sys.executable} -m pip install -q pandas numpy scipy scikit-learn matplotlib seaborn

# %% Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import mannwhitneyu, chi2_contingency
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, accuracy_score
import re
import warnings
from datetime import datetime
warnings.filterwarnings('ignore')

# Set visualization defaults
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("✓ Libraries imported successfully")
print(f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
# %% Define AIDevAnalyzer class
class AIDevAnalyzer:
    """
    Main analyzer class for AIDev dataset research pipeline.
    Implements GQM approach for RQ1, RQ2, and RQ3.
    """

    def __init__(self, output_path='./outputs'):
        """Initialize analyzer with output directory."""
        self.output_path = output_path

        # Core tables
        self.pull_requests = None
        self.repositories = None
        self.users = None
        self.pr_commits = None
        self.pr_commit_details = None
        self.pr_reviews = None
        self.pr_comments = None
        self.pr_task_type = None
        self.related_issues = None
        self.issues = None
        self.pr_timeline = None

        # Human baseline
        self.human_pr = None
        self.human_pr_task_type = None

        # Analysis results
        self.metrics_df = None
        self.combined_df = None

        print("=" * 70)
        print("AIDevAnalyzer Initialized")
        print("=" * 70)
        print(f"Output path: {self.output_path}\n")

    def load_data(self):
        """Load AIDev-pop dataset and human baseline from Hugging Face."""
        print("\n" + "=" * 70)
        print("STEP 1: LOADING DATASET")
        print("=" * 70)

        path = "hf://datasets/hao-li/AIDev/"

        try:
            print("\n[1/12] Loading pull_request (pop)...")
            self.pull_requests = pd.read_parquet(path + 'pull_request.parquet')

            print("[2/12] Loading repository (pop)...")
            self.repositories = pd.read_parquet(path + 'repository.parquet')

            print("[3/12] Loading user (pop)...")
            self.users = pd.read_parquet(path + 'user.parquet')

            print("[4/12] Loading pr_commits (pop)...")
            self.pr_commits = pd.read_parquet(path + 'pr_commits.parquet')

            print("[5/12] Loading pr_commit_details (pop)...")
            self.pr_commit_details = pd.read_parquet(path + 'pr_commit_details.parquet')

            print("[6/12] Loading pr_reviews (pop)...")
            self.pr_reviews = pd.read_parquet(path + 'pr_reviews.parquet')

            print("[7/12] Loading pr_comments (pop)...")
            self.pr_comments = pd.read_parquet(path + 'pr_comments.parquet')

            print("[8/12] Loading pr_task_type (pop)...")
            self.pr_task_type = pd.read_parquet(path + 'pr_task_type.parquet')

            print("[9/12] Loading related_issue (pop)...")
            self.related_issues = pd.read_parquet(path + 'related_issue.parquet')

            print("[10/12] Loading issue (pop)...")
            self.issues = pd.read_parquet(path + 'issue.parquet')

            print("[11/12] Loading pr_timeline (pop)...")
            self.pr_timeline = pd.read_parquet(path + 'pr_timeline.parquet')

            print("[12/12] Loading human_pull_request (base)...")
            self.human_pr = pd.read_parquet(path + 'human_pull_request.parquet')
            self.human_pr_task_type = pd.read_parquet(path + 'human_pr_task_type.parquet')

            print("\n✓ All tables loaded successfully!")
            self._print_dataset_overview()

        except Exception as e:
            print(f"\n✗ Error loading data: {str(e)}")
            raise

    def _print_dataset_overview(self):
        """Print summary statistics of loaded dataset."""
        print("\n" + "-" * 70)
        print("DATASET OVERVIEW")
        print("-" * 70)
        print(f"Agentic PRs (pop): {len(self.pull_requests):,}")
        print(f"Human PRs (base): {len(self.human_pr):,}")
        print(f"Repositories: {len(self.repositories):,}")
        print(f"Users: {len(self.users):,}")
        print(f"Commits: {len(self.pr_commits):,}")
        print(f"Commit Details: {len(self.pr_commit_details):,}")
        print(f"Reviews: {len(self.pr_reviews):,}")
        print(f"Comments: {len(self.pr_comments):,}")
        print("-" * 70)

    def apply_inclusion_exclusion_criteria(self, min_stars=500):
        """
        Apply inclusion/exclusion criteria to create filtered dataset.

        Inclusion:
        - Repos with >= min_stars
        - PRs from Jan 2025 - June 2025
        - Complete metadata (commits available)
        - Clear author attribution

        Exclusion:
        - Incomplete metadata
        - Non-agent bots
        - Ambiguous attribution
        """
        print("\n" + "=" * 70)
        print("STEP 2: APPLYING INCLUSION/EXCLUSION CRITERIA")
        print("=" * 70)

        # Filter Agentic PRs
        initial_agent = len(self.pull_requests)
        print(f"\nInitial Agentic PRs: {initial_agent:,}")

        # Confirm min stars
        high_star_repos = self.repositories[
            self.repositories['stars'] >= min_stars
        ]['id'].values
        self.pull_requests = self.pull_requests[
            self.pull_requests['repo_id'].isin(high_star_repos)
        ]
        print(f"  → After star filter: {len(self.pull_requests):,} PRs")

        # Date filter
        self.pull_requests['created_at'] = pd.to_datetime(self.pull_requests['created_at'])
        date_mask = (
            (self.pull_requests['created_at'] >= '2025-01-01') &
            (self.pull_requests['created_at'] <= '2025-06-22')
        )
        self.pull_requests = self.pull_requests[date_mask]
        print(f"  → After date filter: {len(self.pull_requests):,} PRs")

        # Complete metadata
        complete_prs = self.pr_commits['pr_id'].unique()
        self.pull_requests = self.pull_requests[
            self.pull_requests['id'].isin(complete_prs)
        ]
        print(f"  → After completeness filter: {len(self.pull_requests):,} PRs")

        # Filter Human PRs to same repos
        initial_human = len(self.human_pr)
        print(f"\nInitial Human PRs: {initial_human:,}")


        self.human_pr['created_at'] = pd.to_datetime(self.human_pr['created_at'])
        human_date_mask = (
            (self.human_pr['created_at'] >= '2025-01-01') &
            (self.human_pr['created_at'] <= '2025-06-22')
        )
        self.human_pr = self.human_pr[human_date_mask]
        print(f"  → After filters: {len(self.human_pr):,} PRs")

        # Combine datasets
        self.pull_requests['author_type'] = 'Agent'
        self.human_pr['author_type'] = 'Human'
        self.human_pr['agent'] = 'Human'

        self.combined_df = pd.concat(
            [self.pull_requests, self.human_pr],
            ignore_index=True
        )

        print(f"\n✓ Final combined dataset: {len(self.combined_df):,} PRs")
        print(f"  - Agent: {len(self.pull_requests):,} ({len(self.pull_requests)/len(self.combined_df)*100:.1f}%)")
        print(f"  - Human: {len(self.human_pr):,} ({len(self.human_pr)/len(self.combined_df)*100:.1f}%)")

    def calculate_metrics(self):
        """Calculate all metrics for RQ1, RQ2, RQ3."""
        print("\n" + "=" * 70)
        print("STEP 3: CALCULATING METRICS")
        print("=" * 70)

        # Merge task types
        print("\nMerging task types...")
        agent_tasks = self.pr_task_type.copy()
        agent_tasks['author_type'] = 'Agent'
        human_tasks = self.human_pr_task_type.copy()
        human_tasks['author_type'] = 'Human'
        all_tasks = pd.concat([agent_tasks, human_tasks], ignore_index=True)

        # Merge with main dataset
        self.metrics_df = self.combined_df.merge(
            all_tasks[['id', 'agent', 'type', 'author_type']],
            on=['id', 'agent', 'author_type'],
            how='left'
        )

        self.metrics_df['repo_id'] = pd.to_numeric(self.metrics_df['repo_id'], errors='coerce').astype('Int64')
        self.repositories['id'] = pd.to_numeric(self.repositories['id'], errors='coerce').astype('Int64')

        # Keep all PRs, even if repo metadata is missing
        self.metrics_df = self.metrics_df.merge(
            self.repositories[['id', 'language', 'stars', 'forks']],
            left_on='repo_id',
            right_on='id',
            how='left',
            suffixes=('', '_repo')
        )

        # Preprocessing timestamps
        print("Processing timestamps...")
        self.metrics_df['created_at'] = pd.to_datetime(self.metrics_df['created_at'])
        self.metrics_df['closed_at'] = pd.to_datetime(self.metrics_df['closed_at'], errors='coerce')
        self.metrics_df['merged_at'] = pd.to_datetime(self.metrics_df['merged_at'], errors='coerce')

        # Aggregate commit details per PR
        print("Aggregating commit details...")
        commit_agg = self.pr_commit_details.groupby('pr_id').agg({
            'additions': 'sum',
            'deletions': 'sum',
            'filename': 'nunique',
            'changes': 'sum'
        }).reset_index().rename(columns={'filename': 'files_touched'})

        self.metrics_df = self.metrics_df.merge(
            commit_agg,
            left_on='id',
            right_on='pr_id',
            how='left'
        )

        human_mask = self.metrics_df['author_type'] == 'Human'
        self.metrics_df.loc[human_mask, ['additions', 'deletions', 'files_touched']] = \
            self.metrics_df.loc[human_mask, ['additions', 'deletions', 'files_touched']].fillna(0)


        # Fill NaN values
        self.metrics_df.fillna({
            'additions': 0,
            'deletions': 0,
            'files_touched': 0,
            'changes': 0
        }, inplace=True)

        # RQ1 Metrics: Code Quality
        print("Calculating RQ1 metrics (code quality)...")
        self.metrics_df['loc_changed'] = self.metrics_df['additions'] + self.metrics_df['deletions']
        self.metrics_df['add_del_ratio'] = np.where(
            self.metrics_df['deletions'] > 0,
            self.metrics_df['additions'] / self.metrics_df['deletions'],
            self.metrics_df['additions']
        )

        # Complexity score (proxy)
        self.metrics_df['complexity_score'] = (
            self.metrics_df['additions'] * 0.1 +
            self.metrics_df['files_touched'] * 2
        )

        # Change dispersion
        self.metrics_df['change_dispersion'] = np.where(
            self.metrics_df['loc_changed'] > 0,
            self.metrics_df['files_touched'] / (self.metrics_df['loc_changed'] + 1),
            0
        )

        # Test file detection (basic heuristic)
        test_keywords = ['test', 'spec', '__tests__', '.test.', '.spec.']
        def has_test_files(pr_id):
            if pr_id not in self.pr_commit_details['pr_id'].values:
                return 0
            files = self.pr_commit_details[
                self.pr_commit_details['pr_id'] == pr_id
            ]['filename'].values
            return int(any(any(kw in str(f).lower() for kw in test_keywords) for f in files))

        print("  - Detecting test files...")
        self.metrics_df['has_tests'] = self.metrics_df['id'].apply(has_test_files)

        # RQ2 Metrics: Maintenance Effort
        print("Calculating RQ2 metrics (maintenance effort)...")
        self.metrics_df['is_accepted'] = (~self.metrics_df['merged_at'].isna()).astype(int)
        self.metrics_df['is_closed'] = (~self.metrics_df['closed_at'].isna()).astype(int)

        # Review time (hours)
        self.metrics_df['review_time_hours'] = (
            self.metrics_df['closed_at'] - self.metrics_df['created_at']
        ).dt.total_seconds() / 3600
        self.metrics_df['review_time_hours'] = self.metrics_df['review_time_hours'].fillna(0)

        # Review and comment counts
        review_counts = self.pr_reviews.groupby('pr_id').size().reset_index(name='num_reviews')
        comment_counts = self.pr_comments.groupby('pr_id').size().reset_index(name='num_comments_total')

        self.metrics_df = self.metrics_df.merge(review_counts, left_on='id', right_on='pr_id', how='left', suffixes=('', '_rev_dup'))
        self.metrics_df = self.metrics_df.merge(comment_counts, left_on='id', right_on='pr_id', how='left', suffixes=('', '_com_dup'))

        # Combine counts safely
        self.metrics_df['num_comments'] = (
            self.metrics_df['num_reviews'].fillna(0) +
            self.metrics_df['num_comments_total'].fillna(0)
        )

        # Linked issues (proxy for bug fixes)
        issue_links = self.related_issues.groupby('pr_id').size().reset_index(name='linked_issues')
        self.metrics_df = self.metrics_df.merge(issue_links, left_on='id', right_on='pr_id', how='left')
        self.metrics_df['linked_issues'] = self.metrics_df['linked_issues'].fillna(0)

        # Bug-fix indicator
        self.metrics_df['is_bug_fix'] = (
            self.metrics_df['type'].isin(['fix', 'perf'])
        ).astype(int)

        # RQ3 Metrics: PR Characteristics
        print("Calculating RQ3 metrics (success predictors)...")
        self.metrics_df['title_length'] = self.metrics_df['title'].str.len().fillna(0)
        self.metrics_df['body_length'] = self.metrics_df['body'].str.len().fillna(0)
        self.metrics_df['has_description'] = (self.metrics_df['body_length'] > 0).astype(int)

        # Clean up
        self.metrics_df.drop(columns=['pr_id'], errors='ignore', inplace=True)

        print(f"\n✓ Metrics calculated for {len(self.metrics_df):,} PRs")
        print(f"  Total features: {len(self.metrics_df.columns)}")

    def analyze_rq1(self):
        """RQ1: Structural code quality differences."""
        print("\n" + "=" * 70)
        print("ANALYZING RQ1: STRUCTURAL CODE QUALITY")
        print("=" * 70)

        agent = self.metrics_df[self.metrics_df['author_type'] == 'Agent']
        human = self.metrics_df[self.metrics_df['author_type'] == 'Human']

        # Statistical tests
        results = []

        print("\n[RQ1.1] Code Complexity Comparison:")
        for metric in ['loc_changed', 'files_touched', 'additions', 'deletions',
                       'complexity_score', 'change_dispersion']:
            stat, p = mannwhitneyu(agent[metric].dropna(), human[metric].dropna())

            # Cliff's Delta
            n1, n2 = len(agent[metric].dropna()), len(human[metric].dropna())
            delta = (stat - n1 * n2 / 2) / (n1 * n2)

            results.append({
                'Metric': metric,
                'Agent_Median': agent[metric].median(),
                'Human_Median': human[metric].median(),
                'U_Statistic': stat,
                'p_value': p,
                'Cliff_Delta': delta,
                'Significant': p < 0.05
            })

            print(f"  {metric}: Agent={agent[metric].median():.1f}, "
                  f"Human={human[metric].median():.1f}, p={p:.4f}")

        print("\n[RQ1.2] Testing Practices:")
        # Chi-square for has_tests
        contingency = pd.crosstab(
            self.metrics_df['author_type'],
            self.metrics_df['has_tests']
        )
        chi2, p, dof, expected = chi2_contingency(contingency)

        agent_test_pct = (agent['has_tests'].sum() / len(agent)) * 100
        human_test_pct = (human['has_tests'].sum() / len(human)) * 100

        print(f"  Agent with tests: {agent_test_pct:.1f}%")
        print(f"  Human with tests: {human_test_pct:.1f}%")
        print(f"  Chi-square: χ²={chi2:.2f}, p={p:.4f}")

        results.append({
            'Metric': 'has_tests',
            'Agent_Median': agent_test_pct,
            'Human_Median': human_test_pct,
            'U_Statistic': chi2,
            'p_value': p,
            'Cliff_Delta': None,
            'Significant': p < 0.05
        })

        print("\n[RQ1.3] Task Type Distribution:")
        task_dist = pd.crosstab(
            self.metrics_df['author_type'],
            self.metrics_df['type'],
            normalize='index'
        ) * 100
        print(task_dist.round(1))

        # Save results
        rq1_df = pd.DataFrame(results)
        rq1_df.to_csv(f'{self.output_path}rq1_results.csv', index=False)
        print(f"\n✓ RQ1 results saved to {self.output_path}rq1_results.csv")

        # Visualizations
        self._visualize_rq1()

    def _visualize_rq1(self):
        """Generate RQ1 visualizations."""
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))

        # LOC Changed
        sns.boxplot(data=self.metrics_df, x='author_type', y='loc_changed', ax=axes[0,0])
        axes[0,0].set_title('Lines of Code Changed')
        axes[0,0].set_ylim(0, 500)

        # Files Touched
        sns.boxplot(data=self.metrics_df, x='author_type', y='files_touched', ax=axes[0,1])
        axes[0,1].set_title('Files Modified per PR')
        axes[0,1].set_ylim(0, 20)

        # Complexity Score
        sns.boxplot(data=self.metrics_df, x='author_type', y='complexity_score', ax=axes[0,2])
        axes[0,2].set_title('Complexity Score')
        axes[0,2].set_ylim(0, 100)

        # Testing Rate
        test_rates = self.metrics_df.groupby('author_type')['has_tests'].mean() * 100
        test_rates.plot(kind='bar', ax=axes[1,0])
        axes[1,0].set_title('PRs with Tests (%)')
        axes[1,0].set_ylabel('Percentage')
        axes[1,0].set_xlabel('Author Type')

        # Task Type Distribution
        task_counts = pd.crosstab(
            self.metrics_df['author_type'],
            self.metrics_df['type']
        )
        task_counts.T.plot(kind='bar', ax=axes[1,1])
        axes[1,1].set_title('Task Type Distribution')
        axes[1,1].legend(title='Author Type')
        axes[1,1].set_xlabel('Task Type')

        # Change Dispersion
        sns.boxplot(data=self.metrics_df, x='author_type', y='change_dispersion', ax=axes[1,2])
        axes[1,2].set_title('Change Dispersion')

        plt.tight_layout()
        plt.savefig(f'{self.output_path}rq1_visualizations.png', dpi=300, bbox_inches='tight')
        print(f"✓ Visualizations saved to {self.output_path}rq1_visualizations.png")
        plt.close()

    def analyze_rq2(self):
        """RQ2: Post-merge maintenance effort."""
        print("\n" + "=" * 70)
        print("ANALYZING RQ2: MAINTENANCE EFFORT")
        print("=" * 70)

        agent = self.metrics_df[self.metrics_df['author_type'] == 'Agent']
        human = self.metrics_df[self.metrics_df['author_type'] == 'Human']

        results = []

        print("\n[RQ2.1] Acceptance Rates:")
        # Chi-square for acceptance
        contingency = pd.crosstab(
            self.metrics_df['author_type'],
            self.metrics_df['is_accepted']
        )
        chi2, p, dof, expected = chi2_contingency(contingency)

        agent_accept = (agent['is_accepted'].sum() / len(agent)) * 100
        human_accept = (human['is_accepted'].sum() / len(human)) * 100

        print(f"  Agent acceptance: {agent_accept:.1f}%")
        print(f"  Human acceptance: {human_accept:.1f}%")
        print(f"  Difference: {agent_accept - human_accept:.1f} pp")
        print(f"  Chi-square: χ²={chi2:.2f}, p={p:.4f}")

        results.append({
            'Metric': 'acceptance_rate',
            'Agent_Value': agent_accept,
            'Human_Value': human_accept,
            'Test_Statistic': chi2,
            'p_value': p
        })

        print("\n[RQ2.2] Review Dynamics:")
        # Review time (for closed PRs)
        closed_metrics = self.metrics_df[self.metrics_df['is_closed'] == 1]
        agent_closed = closed_metrics[closed_metrics['author_type'] == 'Agent']
        human_closed = closed_metrics[closed_metrics['author_type'] == 'Human']

        stat, p = mannwhitneyu(
            agent_closed['review_time_hours'].dropna(),
            human_closed['review_time_hours'].dropna()
        )

        print(f"  Agent median review time: {agent_closed['review_time_hours'].median():.1f} hours")
        print(f"  Human median review time: {human_closed['review_time_hours'].median():.1f} hours")
        print(f"  Mann-Whitney U: U={stat:.0f}, p={p:.4f}")

        results.append({
            'Metric': 'review_time_hours',
            'Agent_Value': agent_closed['review_time_hours'].median(),
            'Human_Value': human_closed['review_time_hours'].median(),
            'Test_Statistic': stat,
            'p_value': p
        })

        print("\n[RQ2.3] Review Activity:")

        # Use num_comments we already calculated (reviews + comments)
        agent_activity = agent['num_comments'].dropna()
        human_activity = human['num_comments'].dropna()

        if len(agent_activity) > 0 and len(human_activity) > 0:
            print(f" Agent median review activity: {agent['num_comments'].median():.1f}")
            print(f" Human median review activity: {human['num_comments'].median():.1f}")
            stat, p = mannwhitneyu(agent_activity, human_activity)
            print(f"  Mann-Whitney U: U={stat:.0f}, p={p:.4f}")
        else:
            print("  Insufficient data in one group for review activity comparison")

        results.append({
            'Metric': 'review_activity',
            'Agent_Value': agent['num_comments'].median(),
            'Human_Value': human['num_comments'].median(),
            'Test_Statistic': stat,
            'p_value': p
        })

        # Save results
        rq2_df = pd.DataFrame(results)
        rq2_df.to_csv(f'{self.output_path}rq2_results.csv', index=False)
        print(f"\n✓ RQ2 results saved to {self.output_path}rq2_results.csv")

        # Visualizations
        self._visualize_rq2()

    def _visualize_rq2(self):
        """Generate RQ2 visualizations."""
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))

        # Acceptance rates
        accept_rates = self.metrics_df.groupby('author_type')['is_accepted'].mean() * 100
        accept_rates.plot(kind='bar', ax=axes[0,0], color=['#ff6b6b', '#4ecdc4'])
        axes[0,0].set_title('PR Acceptance Rate (%)')
        axes[0,0].set_ylabel('Acceptance Rate (%)')
        axes[0,0].set_xlabel('Author Type')
        axes[0,0].axhline(y=50, color='gray', linestyle='--', alpha=0.5)

        # Review time distribution (log scale for clarity)
        closed_df = self.metrics_df[self.metrics_df['is_closed'] == 1]
        sns.boxplot(data=closed_df, x='author_type', y='review_time_hours', ax=axes[0,1])
        axes[0,1].set_title('Review Time Distribution (Closed PRs)')
        axes[0,1].set_ylabel('Hours')
        axes[0,1].set_yscale('log')

        # Review activity
        sns.boxplot(data=self.metrics_df, x='author_type', y='num_comments', ax=axes[1,0])
        axes[1,0].set_title('Total Review Activity (Comments + Reviews)')
        axes[1,0].set_ylabel('Count')
        axes[1,0].set_ylim(0, 20)

        # Acceptance by agent
        agent_accept = self.metrics_df[
            self.metrics_df['author_type'] == 'Agent'
        ].groupby('agent')['is_accepted'].mean() * 100
        agent_accept.plot(kind='barh', ax=axes[1,1])
        axes[1,1].set_title('Acceptance Rate by Agent')
        axes[1,1].set_xlabel('Acceptance Rate (%)')

        plt.tight_layout()
        plt.savefig(f'{self.output_path}rq2_visualizations.png', dpi=300, bbox_inches='tight')
        print(f"✓ Visualizations saved to {self.output_path}rq2_visualizations.png")
        plt.close()

    def analyze_rq3(self):
        """RQ3: Success predictors using ML."""
        print("\n" + "=" * 70)
        print("ANALYZING RQ3: SUCCESS PREDICTORS")
        print("=" * 70)

        # Prepare features for ML
        print("\nPreparing features...")

        # Select features
        feature_cols = [
            'loc_changed', 'files_touched', 'additions', 'deletions',
            'complexity_score', 'change_dispersion', 'has_tests',
            'title_length', 'body_length', 'has_description',
            'stars', 'is_bug_fix', 'linked_issues'
        ]

        # Encode categorical features
        ml_df = self.metrics_df.copy()
        ml_df['language_encoded'] = ml_df['language'].astype('category').cat.codes
        ml_df['type_encoded'] = ml_df['type'].astype('category').cat.codes

        feature_cols.extend(['language_encoded', 'type_encoded'])

        # Prepare X and y
        X = ml_df[feature_cols].fillna(0)
        y = ml_df['is_accepted']

        # Remove rows with missing target
        valid_idx = y.notna()
        X = X[valid_idx]
        y = y[valid_idx]

        print(f"  Features: {len(feature_cols)}")
        print(f"  Samples: {len(X):,}")
        print(f"  Positive class: {y.sum():,} ({y.mean()*100:.1f}%)")

        # Train-test split
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.3, random_state=42, stratify=y
        )

        # Train Random Forest
        print("\nTraining Random Forest classifier...")
        rf = RandomForestClassifier(
            n_estimators=100,
            max_depth=10,
            min_samples_split=5,
            class_weight='balanced',
            random_state=42,
            n_jobs=-1
        )

        rf.fit(X_train, y_train)

        # Predictions
        y_pred = rf.predict(X_test)
        y_pred_proba = rf.predict_proba(X_test)[:, 1]

        # Evaluation
        print("\n" + "-" * 70)
        print("MODEL PERFORMANCE")
        print("-" * 70)
        print(classification_report(y_test, y_pred, target_names=['Rejected', 'Accepted']))

        accuracy = accuracy_score(y_test, y_pred)
        roc_auc = roc_auc_score(y_test, y_pred_proba)

        print(f"\nAccuracy: {accuracy:.3f}")
        print(f"ROC-AUC: {roc_auc:.3f}")

        # Feature importance
        print("\n" + "-" * 70)
        print("FEATURE IMPORTANCE")
        print("-" * 70)

        importance_df = pd.DataFrame({
            'Feature': feature_cols,
            'Importance': rf.feature_importances_
        }).sort_values('Importance', ascending=False)

        print(importance_df.head(10).to_string(index=False))

        # Save results
        importance_df.to_csv(f'{self.output_path}rq3_feature_importance.csv', index=False)

        # Cross-validation
        print("\nPerforming 5-fold cross-validation...")
        cv_scores = cross_val_score(rf, X, y, cv=5, scoring='roc_auc', n_jobs=-1)
        print(f"  CV ROC-AUC: {cv_scores.mean():.3f} (+/- {cv_scores.std() * 2:.3f})")

        # Save model performance
        results = {
            'Metric': ['Accuracy', 'ROC-AUC', 'CV ROC-AUC Mean', 'CV ROC-AUC Std'],
            'Value': [accuracy, roc_auc, cv_scores.mean(), cv_scores.std()]
        }
        pd.DataFrame(results).to_csv(f'{self.output_path}rq3_model_performance.csv', index=False)

        print(f"\n✓ RQ3 results saved")

        # Visualizations
        self._visualize_rq3(importance_df, y_test, y_pred)

    def _visualize_rq3(self, importance_df, y_test, y_pred):
        """Generate RQ3 visualizations."""
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))

        # Feature importance (top 10)
        top_features = importance_df.head(10)
        axes[0,0].barh(top_features['Feature'], top_features['Importance'])
        axes[0,0].set_xlabel('Importance')
        axes[0,0].set_title('Top 10 Feature Importances')
        axes[0,0].invert_yaxis()

        # Confusion matrix
        cm = confusion_matrix(y_test, y_pred)
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0,1])
        axes[0,1].set_xlabel('Predicted')
        axes[0,1].set_ylabel('Actual')
        axes[0,1].set_title('Confusion Matrix')
        axes[0,1].set_xticklabels(['Rejected', 'Accepted'])
        axes[0,1].set_yticklabels(['Rejected', 'Accepted'])

        # Acceptance by task type
        accept_by_task = self.metrics_df.groupby('type')['is_accepted'].mean() * 100
        accept_by_task.sort_values(ascending=False).plot(kind='barh', ax=axes[1,0])
        axes[1,0].set_xlabel('Acceptance Rate (%)')
        axes[1,0].set_title('Acceptance Rate by Task Type')

        # PR size vs acceptance
        self.metrics_df['size_category'] = pd.cut(
            self.metrics_df['loc_changed'],
            bins=[0, 50, 200, 1000, float('inf')],
            labels=['Small (<50)', 'Medium (50-200)', 'Large (200-1000)', 'XLarge (>1000)']
        )
        size_accept = self.metrics_df.groupby('size_category')['is_accepted'].mean() * 100
        size_accept.plot(kind='bar', ax=axes[1,1])
        axes[1,1].set_xlabel('PR Size')
        axes[1,1].set_ylabel('Acceptance Rate (%)')
        axes[1,1].set_title('Acceptance Rate by PR Size')
        axes[1,1].tick_params(axis='x', rotation=45)

        plt.tight_layout()
        plt.savefig(f'{self.output_path}rq3_visualizations.png', dpi=300, bbox_inches='tight')
        print(f"✓ Visualizations saved to {self.output_path}rq3_visualizations.png")
        plt.close()

    def generate_summary_report(self):
        """Generate executive summary."""
        print("\n" + "=" * 70)
        print("GENERATING EXECUTIVE SUMMARY")
        print("=" * 70)

        summary = []
        summary.append("=" * 70)
        summary.append("AIDEV RESEARCH ANALYSIS - EXECUTIVE SUMMARY")
        summary.append("=" * 70)
        summary.append(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

        # Dataset
        summary.append("DATASET SUMMARY")
        summary.append("-" * 70)
        summary.append(f"Total PRs: {len(self.metrics_df):,}")

        agent = self.metrics_df[self.metrics_df['author_type'] == 'Agent']
        human = self.metrics_df[self.metrics_df['author_type'] == 'Human']

        summary.append(f"Agent PRs: {len(agent):,} ({len(agent)/len(self.metrics_df)*100:.1f}%)")
        summary.append(f"Human PRs: {len(human):,} ({len(human)/len(self.metrics_df)*100:.1f}%)")
        summary.append(f"Repositories: {self.metrics_df['repo_id'].nunique():,}")
        summary.append(f"Languages: {self.metrics_df['language'].nunique()}")

        # Key Findings
        summary.append("\n\nKEY FINDINGS")
        summary.append("-" * 70)

        # RQ1
        summary.append("\n[RQ1] Structural Code Quality:")
        summary.append(f"  • Agent median LOC: {agent['loc_changed'].median():.0f} vs Human: {human['loc_changed'].median():.0f}")
        summary.append(f"  • Agent median files: {agent['files_touched'].median():.0f} vs Human: {human['files_touched'].median():.0f}")
        summary.append(f"  • Agent test inclusion: {(agent['has_tests'].sum()/len(agent)*100):.1f}% vs Human: {(human['has_tests'].sum()/len(human)*100):.1f}%")

        # RQ2
        summary.append("\n[RQ2] Maintenance Effort:")
        summary.append(f"  • Agent acceptance: {(agent['is_accepted'].sum()/len(agent)*100):.1f}% vs Human: {(human['is_accepted'].sum()/len(human)*100):.1f}%")

        closed_agent = agent[agent['is_closed'] == 1]
        closed_human = human[human['is_closed'] == 1]
        summary.append(f"  • Agent review time: {closed_agent['review_time_hours'].median():.1f}h vs Human: {closed_human['review_time_hours'].median():.1f}h")
        summary.append(f" • Agent review activity: {agent['num_comments'].median():.1f} vs Human: {human['num_comments'].median():.1f}")

        # RQ3
        summary.append("\n[RQ3] Success Predictors:")
        summary.append("  • See feature importance CSV for detailed rankings")
        summary.append(f"  • Model accuracy: See RQ3 results")

        summary.append("\n" + "=" * 70)
        summary.append("RECOMMENDATIONS")
        summary.append("=" * 70)
        summary.append("1. Developers should focus on test inclusion when using AI agents")
        summary.append("2. Review processes need adaptation for AI-generated code")
        summary.append("3. AI agents excel at documentation but need improvement in complex tasks")
        summary.append("4. Tool designers should optimize for acceptance, not just speed")
        summary.append("\n" + "=" * 70)

        summary_text = "\n".join(summary)
        print(summary_text)

        with open(f'{self.output_path}executive_summary.txt', 'w') as f:
            f.write(summary_text)

        print(f"\n✓ Summary saved to {self.output_path}executive_summary.txt")

In [9]:
# %% Main execution pipeline
def main():
    """Execute complete research pipeline."""
    print("\n" + "=" * 70)
    print("AIDEV ANALYSIS PIPELINE - START")
    print("=" * 70)
    print(f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

    try:
        # Initialize
        analyzer = AIDevAnalyzer(output_path='./outputs/')

        # Create output directory
        import os
        os.makedirs(analyzer.output_path, exist_ok=True)

        # Execute pipeline
        analyzer.load_data()
        analyzer.apply_inclusion_exclusion_criteria(min_stars=500)
        analyzer.calculate_metrics()
        analyzer.analyze_rq1()
        analyzer.analyze_rq2()
        analyzer.analyze_rq3()
        analyzer.generate_summary_report()

        print("\n" + "=" * 70)
        print("PIPELINE COMPLETED SUCCESSFULLY!")
        print("=" * 70)
        print(f"\nAll outputs saved to: {analyzer.output_path}")
        print("\nGenerated files:")
        print("  - filtered_dataset.csv")
        print("  - rq1_results.csv")
        print("  - rq1_visualizations.png")
        print("  - rq2_results.csv")
        print("  - rq2_visualizations.png")
        print("  - rq3_feature_importance.csv")
        print("  - rq3_model_performance.csv")
        print("  - rq3_visualizations.png")
        print("  - executive_summary.txt")

        return 0

    except Exception as e:
        print(f"\n✗ ERROR: {str(e)}")
        import traceback
        traceback.print_exc()
        return 1

# %% Run pipeline
if __name__ == "__main__":
    exit_code = main()

✓ Libraries imported successfully
Timestamp: 2026-01-04 00:22:24

AIDEV ANALYSIS PIPELINE - START
Timestamp: 2026-01-04 00:22:24
AIDevAnalyzer Initialized
Output path: ./outputs/


STEP 1: LOADING DATASET

[1/12] Loading pull_request (pop)...
[2/12] Loading repository (pop)...
[3/12] Loading user (pop)...
[4/12] Loading pr_commits (pop)...
[5/12] Loading pr_commit_details (pop)...
[6/12] Loading pr_reviews (pop)...
[7/12] Loading pr_comments (pop)...
[8/12] Loading pr_task_type (pop)...
[9/12] Loading related_issue (pop)...
[10/12] Loading issue (pop)...
[11/12] Loading pr_timeline (pop)...
[12/12] Loading human_pull_request (base)...

✓ All tables loaded successfully!

----------------------------------------------------------------------
DATASET OVERVIEW
----------------------------------------------------------------------
Agentic PRs (pop): 33,596
Human PRs (base): 6,618
Repositories: 2,807
Users: 1,796
Commits: 88,576
Commit Details: 711,923
Reviews: 28,875
Comments: 39,122
------